In [1]:
# Installing necessary libraries

!pip install sentence-transformers pandas googletrans==4.0.0-rc1 datasets transformers gradio


  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of gradio to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of gradio to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looking at multiple versions of fastapi to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5

In [2]:
# Importing required libraries

import pandas as pd

from sentence_transformers import SentenceTransformer, util

from googletrans import Translator

from transformers import pipeline

import gradio as gr


In [3]:
# Initializing models and translator

translator = Translator()  # Initializing the Google Translator to translate text

english_pipeline = pipeline("question-answering", model="distilbert-base-uncased-distilled-squad")  # Initializing English question-answering pipeline using distilBERT

multilingual_pipeline = pipeline("question-answering", model="xlm-roberta-base")  # Initializing Multilingual question-answering pipeline using XLM-RoBERTa

sentence_model = SentenceTransformer("all-MiniLM-L6-v2")  # Initializing the sentence transformer model to generate embeddings




config.json:   0%|          | 0.00/451 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/265M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [4]:
# Defining dataset with questions and contexts

data = [

    {"context": "Artificial Intelligence (AI) refers to the simulation of human intelligence in machines.",

     "question": "What does AI refer to?"},

    {"context": "The Eiffel Tower is a wrought-iron lattice tower on the Champ de Mars in Paris, France.",

     "question": "Where is the Eiffel Tower located?"},

    {"context": "The Mona Lisa is a portrait painting by the Italian artist Leonardo da Vinci.",

     "question": "Who painted the Mona Lisa?"},

    {"context": "The Great Wall of China is a series of fortifications made of various materials.",

     "question": "What is the Great Wall of China made of?"},

    {"context": "The speed of light in a vacuum is approximately 299,792 kilometers per second.",

     "question": "What is the speed of light?"}

]



# Translating dataset to Bangla

for row in data:

    row['context_bn'] = translator.translate(row['context'], src='en', dest='bn').text  # Translating context to Bangla

    row['question_bn'] = translator.translate(row['question'], src='en', dest='bn').text  # Translating question to Bangla




In [5]:
# Defining a function to retrieve the most relevant context for a given question

def retrieve_context(question, documents):

    question_embedding = sentence_model.encode(question, convert_to_tensor=True)  # Encoding the question to get its embedding

    document_embeddings = sentence_model.encode(documents, convert_to_tensor=True)  # Encoding the documents (contexts) to get their embeddings

    scores = util.pytorch_cos_sim(question_embedding, document_embeddings)  # Calculating the cosine similarity between the question and the documents

    best_doc_idx = scores.argmax().item()  # Finding the index of the most relevant document

    return documents[best_doc_idx]  # Returning the most relevant document (context)




In [6]:
# Defining a function to generate answers in both English and Bangla

def generate_answers(question, idx):

    context_en = retrieve_context(question, [data[idx]['context']])  # Retrieving the English context

    context_bn = retrieve_context(question, [data[idx]['context_bn']])  # Retrieving the Bangla context



    try:

        en_result = english_pipeline({'context': context_en, 'question': question})  # Generating answer in English using the English pipeline

        en_answer = en_result['answer']  # Extracting the English answer

    except Exception as e:

        en_answer = f"Error: {e}"  # Handling any errors while generating the English answer



    try:

        bn_question = translator.translate(question, src='en', dest='bn').text  # Translating the question to Bangla

        bn_result = multilingual_pipeline({'context': context_bn, 'question': bn_question})  # Generating answer in Bangla using the multilingual pipeline

        bn_answer = bn_result['answer']  # Extracting the Bangla answer

    except Exception as e:

        bn_answer = f"Error: {e}"  # Handling any errors while generating the Bangla answer



    # Returning the answers along with the context in both English and Bangla

    return f"English Context:\n{context_en}\n\nEnglish Answer:\n{en_answer}\n\nBangla Context:\n{context_bn}\n\nBangla Answer:\n{bn_answer}"




In [7]:
# Defining Gradio Interface function

def qa_interface(question, idx):

    idx = int(idx) - 1  # Adjusting the index to be 0-based as Python uses 0-based indexing

    return generate_answers(question, idx)  # Returning the generated answers by calling the generate_answers function




In [8]:
# Defining Gradio input and output components

questions = [f"{i+1}: {row['question']}" for i, row in enumerate(data)]  # Creating a list of questions for the interface




In [9]:
# Creating Gradio interface with input text box for question and number input for dataset index

interface = gr.Interface(

    fn=qa_interface,  # Function that will be called on user input

    inputs=[gr.Textbox(label="Question"), gr.Number(label="Dataset Index (1-based)")],  # Inputs for question and dataset index

    outputs="text",  # Output type: text will be shown in the output area

    title="Question Answering System",  # Title of the app

    description="Provide a question and the dataset index to retrieve the relevant context and generate answers."  # Description of the app functionality

)






IMPORTANT: You are using gradio version 4.19.1, however version 4.44.1 is available, please upgrade.
--------


In [11]:
# Launching the Gradio app

interface.launch()  # Launching the interface to be accessible in the browser

Rerunning server... use `close()` to stop if you need to change `launch()` parameters.
----
Kaggle notebooks require sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Running on public URL: https://31858fa099ba45d10a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
